# Milestone 1 - EDA and Preprocessing data 

- Load dataset
- Explore the dataset and ask atleast 5 questions to give you a better understanding of the data provided to you. 
- Visualise the answer to these 5 questions.
- Cleaing the data
- Observe missing data and comment on why you believe it is missing(MCAR,MAR or MNAR) 
- Observe duplicate data
- Observe outliers
- After observing outliers,missing data and duplicates, handle any unclean data.
- With every change you are making to the data you need to comment on why you used this technique and how has it affected the data(by both showing the change in the data i.e change in number of rows/columns,change in distrubution, etc and commenting on it).
- Data transformation and feature engineering
- Add a new column named 'Week number' and discretisize the data into weeks according to the dates.Tip: Change the datatype of the date feature to datetime type instead of object.
- Encode any categorical feature(s) and comment on why you used this technique and how the data has changed.
- Identify feature(s) which need normalisation and show your reasoning.Then choose a technique to normalise the feature(s) and comment on why you chose this technique.
- Add atleast two more columns which adds more info to the dataset by evaluating specific feature(s). I.E( Column indicating whether the accident was on a weekend or not). 
- For any imputation with arbitrary values or encoding done, you have to store what the value imputed or encoded represents in a new csv file. I.e if you impute a missing value with -1 or 100 you must have a csv file illustrating what -1 and 100 means. Or for instance, if you encode cities with 1,2,3,4,etc what each number represents must be shown in the new csv file.
- Load the new dataset into a csv file.
- **Extremely Important note** - Your code should be as generic as possible and not hard-coded and be able to work with various datasets. Any hard-coded solutions will be severely penalised.
- Bonus: Load the dataset as a parquet file instead of a csv file(Parquet file is a compressed file format).

# 1 - Extraction

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.neighbors import LocalOutlierFactor
from scipy import stats


In [ ]:
pd.set_option('display.max_columns', None)
dataset = 'dataset/1980_Accidents_UK.csv'
df_accidents_1980 = pd.read_csv(dataset, index_col=None)
df_accidents_1980.head(50)


# 2- EDA

**Does accident_index have uniqe values and what is it's relatiion to accident_year and accident_refrence?**

**Is there a relation between accident_severity and number_of_casualties?**

In [ ]:
sns.barplot(x="accident_severity", y="number_of_casualties",
            data=df_accidents_1980)


As shown in the previous graph there is a direct relationship between accident_severity and number_of_casualties.

In [ ]:
sns.barplot(x="accident_severity", y="speed_limit",
            data=df_accidents_1980)


In [ ]:
perc_null = df_accidents_1980.isnull().mean() * 100
print(perc_null)


In [ ]:
df_unique = df_accidents_1980.apply(lambda col: col.unique())
print(df_unique)

# 3 - Cleaning Data

## Observing Missing and duplicate Data

 ### We replace all occurances of 'Data missing or out of range' and -1 with nan to have consistencey

In [ ]:
perc_null = df_accidents_1980.isnull().mean() * 100
perc_null

In [ ]:
df_unique = df_accidents_1980.apply(lambda col: col.unique())
df_unique

In [ ]:
accident_index_unique_counts = len(
    df_accidents_1980["accident_index"].unique())
accident_refernce_unique_counts = len(
    df_accidents_1980["accident_reference"].unique())
number_of_entries = len(df_accidents_1980)
"Number of Unique accident_index: " + \
    str(accident_index_unique_counts), "Number of Unique accident_refrence: " + \
    str(accident_refernce_unique_counts), "Number of total entries:  " + \
    str(number_of_entries)

(np.equal((df_accidents_1980["accident_year"].astype(str) +
          df_accidents_1980["accident_reference"]), df_accidents_1980["accident_index"])).all()

df_accidents_1980["accident_year"].unique()

In [ ]:
num_c_un = df_accidents_1980[df_accidents_1980['first_road_class']
                         == 'C']['first_road_class'].count() + df_accidents_1980[df_accidents_1980['first_road_class']
                                                                                       == 'Unclassified']['first_road_class'].count()
num_missing_road = df_accidents_1980[df_accidents_1980['first_road_number']
                                     == 'first_road_class is C or Unclassified. These roads do not have official numbers so recorded as zero ']['first_road_class'].count()
if num_c_un == num_missing_road:
    print(True)
else:
    print(False)

## Handling Missing data

In [ ]:
df_accidents_1980_clean = df_accidents_1980.replace(
    'Data missing or out of range', np.nan)
df_accidents_1980_clean = df_accidents_1980_clean.replace(
    -1, np.nan)
df_accidents_1980_clean = df_accidents_1980_clean.replace(
    '-1', np.nan)
df_accidents_1980_clean = df_accidents_1980_clean.replace(
    'first_road_class is C or Unclassified. These roads do not have official numbers so recorded as zero ', 0)
df_null = df_accidents_1980_clean.isnull().mean() * 100
df_null

In [ ]:
df_accidents_1980_clean = df_accidents_1980_clean.dropna(
    axis='columns', how='all')

In [ ]:
(np.equal((df_accidents_1980["accident_year"].astype(str) +
          df_accidents_1980["accident_reference"]), df_accidents_1980["accident_index"])).all()  # checking the accident_year concatenated to the accident_reference is equal to the accident_index
df_accidents_1980_clean = df_accidents_1980_clean.drop(
    'accident_index', axis=1)  # dropping the accident_index

df_accidents_1980_clean = df_accidents_1980_clean.set_index(
    'accident_reference')


In [ ]:
df_accidents_1980_clean = df_accidents_1980_clean.dropna(
    axis='index', how='any', subset=['location_easting_osgr', 'location_easting_osgr', 'junction_detail', 'first_road_number', 'light_conditions', 'weather_conditions', 'road_surface_conditions', 'carriageway_hazards'])
#df_accidents_1980_clean = df_accidents_1980_clean.drop('second_road_number',axis=1)



In [ ]:
df_filter = df_accidents_1980_clean[(df_accidents_1980_clean['second_road_class'].notna()) & ~(
    df_accidents_1980_clean['second_road_number'].notna())].index
df_accidents_1980_clean = df_accidents_1980_clean.drop(df_filter)
print((df_accidents_1980_clean[df_accidents_1980_clean['junction_detail']
                         == 'Not at junction or within 20 metres']['junction_detail'].count()) / len(df_accidents_1980_clean.index) * 100)

print(df_null['junction_control'])

In [ ]:
df_filter = df_accidents_1980_clean[((df_accidents_1980_clean['junction_detail'] != 'Not at junction or within 20 metres') & ~(
    df_accidents_1980_clean['junction_control'].notna())) |((df_accidents_1980_clean['junction_detail'].isna()) & (
    df_accidents_1980_clean['junction_control'].notna()))].index
df_accidents_1980_clean = df_accidents_1980_clean.drop(df_filter)
print((df_accidents_1980_clean[df_accidents_1980_clean['junction_detail']
                         == 'Not at junction or within 20 metres']['junction_detail'].count()) / len(df_accidents_1980_clean.index) * 100)

print(df_null['junction_control'])

In [ ]:

data = df_accidents_1980_clean[['pedestrian_crossing_human_control',
                                 'pedestrian_crossing_physical_facilities']].dropna()
data['pedestrian_crossing_physical_facilities'] = data['pedestrian_crossing_physical_facilities'].astype(
    'category')
data['pedestrian_crossing_physical_facilities'] = data['pedestrian_crossing_physical_facilities'].cat.codes

data['pedestrian_crossing_human_control'] = data['pedestrian_crossing_human_control'].astype(
    'category')
data['pedestrian_crossing_human_control'] = data['pedestrian_crossing_human_control'].cat.codes

sns.kdeplot(data["pedestrian_crossing_physical_facilities"])

plt.show()
sns.kdeplot(data["pedestrian_crossing_human_control"])
plt.show()

In [ ]:
data["pedestrian_crossing_human_control"].median()
ind = data[data["pedestrian_crossing_human_control"] == data["pedestrian_crossing_human_control"].median()].index[0]
pedesMedian = df_accidents_1980_clean["pedestrian_crossing_human_control"][ind]
df_accidents_1980_clean["pedestrian_crossing_human_control"] = df_accidents_1980_clean["pedestrian_crossing_human_control"].replace(np.nan, pedesMedian)
df_null = df_accidents_1980_clean.isna().mean() * 100

df_null


In [ ]:
data["pedestrian_crossing_physical_facilities"].median()
ind = data[data["pedestrian_crossing_physical_facilities"] == data["pedestrian_crossing_physical_facilities"].median()].index[0]
pedesMedian = df_accidents_1980_clean["pedestrian_crossing_physical_facilities"][ind]
df_accidents_1980_clean["pedestrian_crossing_physical_facilities"] = df_accidents_1980_clean["pedestrian_crossing_physical_facilities"].replace(np.nan, pedesMedian)
df_null = df_accidents_1980_clean.isna().mean() * 100

df_null

In [ ]:
df_accidents_1980_clean["special_conditions_at_site"].unique()
data = df_accidents_1980_clean[['special_conditions_at_site']].dropna()
data['special_conditions_at_site'] = data['special_conditions_at_site'].astype(
    'category')
data['special_conditions_at_site'] = data['special_conditions_at_site'].cat.codes
sns.kdeplot(data["special_conditions_at_site"])



In [ ]:
data["special_conditions_at_site"].median()
ind = data[data["special_conditions_at_site"] == data["special_conditions_at_site"].median()].index[0]
specMedian = df_accidents_1980_clean["special_conditions_at_site"][ind]
df_accidents_1980_clean["special_conditions_at_site"] = df_accidents_1980_clean["special_conditions_at_site"].replace(np.nan, pedesMedian)
df_null = df_accidents_1980_clean.isna().mean() * 100

df_null

## Findings and conclusions

In [ ]:

data = df_accidents_1980_clean[['road_type']].dropna()
data['road_type'] = data['road_type'].astype(
    'category')
data['road_type'] = data['road_type'].cat.codes
sns.kdeplot(data["road_type"])
data["road_type"].median()
ind = data[data["road_type"] == data["road_type"].median()].index[0]
specMedian = df_accidents_1980_clean["road_type"][ind]
df_accidents_1980_clean["road_type"] = df_accidents_1980_clean["road_type"].replace(np.nan, pedesMedian)
df_accidents_1980_clean["junction_control"] = df_accidents_1980_clean["junction_control"].replace(np.nan, "Non")

df_accidents_1980_clean = df_accidents_1980_clean.drop('second_road_class',axis=1)
df_accidents_1980_clean = df_accidents_1980_clean.drop('second_road_number',axis=1)
df_null = df_accidents_1980_clean.isna().mean() * 100

df_null


## Observing outliers

In [ ]:
plt.boxplot(df_accidents_1980_clean['number_of_vehicles'])

z = np.abs(stats.zscore(df_accidents_1980_clean['number_of_vehicles']))

veh_filtered_entries = z < 3
(np.bitwise_not(veh_filtered_entries).sum() /
 len(df_accidents_1980_clean.index)) * 100


In [ ]:
sns.kdeplot(df_accidents_1980_clean['number_of_vehicles'])
Q1 = df_accidents_1980_clean['number_of_vehicles'].quantile(0.25)
Q3 = df_accidents_1980_clean['number_of_vehicles'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)
cut_off = IQR * 1.5
lower = Q1 - cut_off
upper = Q3 + cut_off
print(lower, upper)
df1 = df_accidents_1980_clean[df_accidents_1980_clean['number_of_vehicles'] > upper]
df2 = df_accidents_1980_clean[df_accidents_1980_clean['number_of_vehicles'] < lower]
print('Percentage of outliers are',
      (df1.shape[0] + df2.shape[0]) / len(df_accidents_1980_clean.index) * 100)


In [ ]:
plt.boxplot(df_accidents_1980_clean['number_of_casualties'])

z = np.abs(stats.zscore(df_accidents_1980_clean['number_of_casualties']))

cas_filtered_entries = z < 4
(np.bitwise_not(cas_filtered_entries).sum() /
 len(df_accidents_1980_clean.index)) * 100


In [ ]:
sns.kdeplot(df_accidents_1980_clean['number_of_casualties'])
Q1 = df_accidents_1980_clean['number_of_casualties'].quantile(0.25)
Q3 = df_accidents_1980_clean['number_of_casualties'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)
cut_off = IQR * 1.5
lower = Q1 - cut_off
upper = Q3 + cut_off
print(lower, upper)
df1 = df_accidents_1980_clean[df_accidents_1980_clean['number_of_casualties'] > upper]
df2 = df_accidents_1980_clean[df_accidents_1980_clean['number_of_casualties'] < lower]
print('Percentage of outliers are',
      (df1.shape[0] + df2.shape[0]) / len(df_accidents_1980_clean.index))


In [ ]:
plt.boxplot(df_accidents_1980_clean['speed_limit'])

z = np.abs(stats.zscore(df_accidents_1980_clean['speed_limit']))

filtered_entries = z < 3
(np.bitwise_not(filtered_entries).sum() / len(df_accidents_1980_clean.index)) * 100


In [ ]:
sns.kdeplot(df_accidents_1980_clean['speed_limit'])
Q1 = df_accidents_1980_clean['speed_limit'].quantile(0.25)
Q3 = df_accidents_1980_clean['speed_limit'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)
cut_off = IQR * 1.5
lower = Q1 - cut_off
upper = Q3 + cut_off
print(lower, upper)
df1 = df_accidents_1980_clean[df_accidents_1980_clean['speed_limit'] > upper]
df2 = df_accidents_1980_clean[df_accidents_1980_clean['speed_limit'] < lower]
print('Percentage of outliers are',
      (df1.shape[0] + df2.shape[0]) / len(df_accidents_1980_clean.index))


## Handling outliers

In [ ]:
veh_med = df_accidents_1980_clean[veh_filtered_entries]['number_of_vehicles'].median(
)
print(veh_med)
print(len(df_accidents_1980_clean.index))
df_accidents_1980_clean['number_of_vehicles'] = df_accidents_1980_clean['number_of_vehicles'].where(
    veh_filtered_entries, other=veh_med)
df_accidents_1980_clean[~veh_filtered_entries]


In [ ]:
cas_med = df_accidents_1980_clean[cas_filtered_entries]['number_of_casualties'].median(
)
print(cas_filtered_entries)
print(len(df_accidents_1980_clean.index))
df_accidents_1980_clean['number_of_casualties'] = df_accidents_1980_clean['number_of_casualties'].where(
    cas_filtered_entries, other=cas_med)
df_accidents_1980_clean[~cas_filtered_entries]


## Findings and conclusions

# 4 - Data transformation

## 4.1 - Discretization

## 4.11 - Findings and conclusions

## 4.2 - Encoding

## 4.22 - Findings and conlcusions

## 4.3 - Normalisation 

## 4.31 - Findings and conclusions

## 4.4 - Adding more columns

## 4.41 - Findings and concluisons

## 4.5 - Csv file for lookup

## 5- Exporting the dataframe to a csv file or parquet